In [ ]:
pip install ucimlrepo numpy pandas matplotlib scikit-learn seaborn autograd torch tabulate xgboost

In [ ]:
try:
    from ucimlrepo import fetch_ucirepo

    # fetch dataset
    support2 = fetch_ucirepo(id=880)

    # data (as pandas dataframes)
    X = support2.data.features
    y = support2.data.targets

    # metadata
    print(support2.metadata)

    # variable information
    print(support2.variables)
except ConnectionError as e:
    import pandas as pd
    print("Unable to fetch dataset. Trying to load from local file.")
    support2 = pd.read_csv("data.csv")
    X = support2.iloc[:,:-3]
    y = support2.iloc[:,-3:]

In [ ]:
# support2.variables.to_csv("support2_codebook.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.concat([X,y],axis=1)
data

In [ ]:
missed_target_data = data[data['sfdm2'].isnull()]
data = data.dropna(subset=['sfdm2'])

In [ ]:
from tabulate import tabulate
def dataUnderstanding(data):
  # Number of the rows and columns
  rows, columns = data.shape
  print('-' * 50)
  print(f"Number of Rows:{rows} | Number of Columns:{columns}")
  print('-' * 50)
  print()

  # List of columns
  columns_df = pd.DataFrame(data.columns, columns=["Column Names"])
  print("List of Features in the dataset:")
  print(tabulate(columns_df, headers='keys', tablefmt='psql', showindex=False))
  print('-' * 50)
  print()

  # Print Data type
  print("Summarized basic information:\n")
  data.info()
  print('-' * 50)
  print()

  # printing all the numerical datatype columns
  numerical_columns = data.select_dtypes(include=['number']).columns.tolist()
  print(f"Printing all the numerical columns --> {numerical_columns}")
  print()
  # printing all the object datatype columns
  object_columns = data.select_dtypes(include=['object']).columns.tolist()
  print(f"Printing all the Object columns --> {object_columns}")
  print('-' * 50)
  print()

  # finding the missing values
  print(f"Finding the number of missing values in all the columns -->\n")
  print(data.isna().sum())
  print('-' * 50)


dataUnderstanding(data)

# Data Preprocessing

## Data Cleaning

### Missing Values

In [ ]:
missing_value_table = data.isnull().sum()
missing_value_proportion = missing_value_table[missing_value_table>0].sort_values(ascending=False) / len(data)
for i in missing_value_proportion.index:
    print("{}: {:.2f}%".format(i,missing_value_proportion[i]*100), f'dtype={data[i].dtype}')

In [ ]:
X_pd = pd.DataFrame(X)
y_pd = pd.DataFrame(y)
X_pd.isnull().sum(), y_pd.isnull().sum()

In [ ]:
for i in data.columns:
  print(f'{i}:{data[i].dtype}')

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

def fill_missing_values_simplified(data: pd.DataFrame) -> pd.DataFrame:
    """
    Fill missing values in a DataFrame using KNNImputer for numerical columns
    and SimpleImputer (most frequent) for categorical columns.

    Parameters:
    -----------
    data : pd.DataFrame
        The input DataFrame containing missing values.

    Returns:
    --------
    pd.DataFrame
        The DataFrame with missing values filled.
    """
    data_filled = data.copy()
    
    # Identify numerical and categorical columns
    numerical_cols = data.select_dtypes(include=['number']).columns.tolist()
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

    # Impute numerical columns with KNNImputer
    if numerical_cols and data_filled[numerical_cols].isnull().sum().sum() > 0:
        knn_imputer = KNNImputer(n_neighbors=13,weights='distance')
        data_filled[numerical_cols] = knn_imputer.fit_transform(data_filled[numerical_cols])

    # Impute categorical columns with the most frequent value
    if categorical_cols and data_filled[categorical_cols].isnull().sum().sum() > 0:
        simple_imputer = SimpleImputer(strategy='most_frequent')
        data_filled[categorical_cols] = simple_imputer.fit_transform(data_filled[categorical_cols])

    return data_filled

# Create a copy to avoid modifying the original data in place
data_filled = data.copy()

# Loop through columns with missing values and apply the simplified filling logic
for col in missing_value_proportion.index:
    # Add a missing value indicator column before filling
    data_filled['missing_' + col] = data_filled[col].isnull().astype(float)

# Apply the simplified imputation to the whole dataframe
data = fill_missing_values_simplified(data_filled)


In [ ]:
data.isnull().sum()

In [ ]:
data

### Outlier Detection

In [ ]:
def outlier_detection(data):
    """
    Detect outliers for numerical and categorical features.
    Returns a DataFrame with outlier flags (1=outlier, 0=normal) for each method.
    """
    results = data.copy()
    outlier_flags = pd.DataFrame(index=results.index)

    # Numerical: IQR method (threshold=3, typical for moderate outlier frequency)
    num_cols = results.select_dtypes(include=["number"]).columns
    # Exclude columns that start with 'missing_'
    num_cols = [col for col in num_cols if not col.startswith('missing_')]
    def iqr_detector(col, threshold=3):
        q1 = col.quantile(0.25)
        q3 = col.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - threshold * iqr
        upper_bound = q3 + threshold * iqr
        return ((col < lower_bound) | (col > upper_bound)).astype(int)
    for col in num_cols:
        outlier_flags[f'iqr_{col}'] = iqr_detector(results[col])

    # Categorical: rare category (threshold=0.0005, i.e., <.1% frequency)
    cat_cols = results.select_dtypes(include=["object", "category"]).columns
    def category_outlier_detector(col, threshold=0.0005):
        freq = col.value_counts(normalize=True)
        rare_categories = freq[freq < threshold].index
        return col.isin(rare_categories).astype(int)
    for col in cat_cols:
        outlier_flags[f'cat_outlier_{col}'] = category_outlier_detector(results[col])

    return outlier_flags

In [ ]:
demo = outlier_detection(data.copy())
demo.describe()

In [ ]:
count = 0
for row in demo.index:
    if demo.loc[row].sum() > 6:
        # print(f"Row {row} is an outlier in the following methods: {demo.loc[row][demo.loc[row] == 1].index.tolist()}")
        pass
    else:
        count += 1
count

In [ ]:
# delete outliers
def remove_outliers(data):
    """
    Remove outliers from the DataFrame.
    Returns a DataFrame with outliers removed.
    """
    crit = outlier_detection(data)
    for row in crit.index:
        if crit.loc[row].sum() > 6:
            data = data.drop(row)
    return data

data_no_outliers = remove_outliers(data.copy())

In [ ]:
# recode the index
data_no_outliers.reset_index(drop=True, inplace=True)
data_no_outliers

## Feature Engineering

In [ ]:
data = data_no_outliers
data["Age_Class"] = pd.cut(data["age"], bins=[0, 40, 65, 79, 130], labels=["Young Adult","Adult", "Senior", "Elderly"])
data["Age_Class"].value_counts()

In [ ]:
data['Risk'] = data['surv2m'] * data['surv2m'] / (data['surv6m']+1e-3)
data['phy_Risk'] = data['prg2m'] * data['prg2m'] / (data['prg6m']+1e-3)
data['short_term_diff'] = data['surv2m'] / data['prg2m']
data['long_term_diff'] = data['surv6m'] / data['prg6m']
# sigmoid scaling, change into [0,1] range and indicate a probability of shouldering a risk for month, which in common sense, is related to sfdm2 or death
data['Risk'] = 1 / (1 + np.exp(-data['Risk']))
data['Risk'].describe()

In [ ]:
# costs are often right skewed, so we can use log transformation
data['charges_log'] = np.log(data['charges'] + 1e-3)  # Adding a small constant to avoid log(0)
data['totcst_log'] = np.log(data['totcst'] + 1e-3)  # Adding a small constant to avoid log(0)
data['totcst_log'] = 5*(data['totcst_log']-min(data['totcst_log'])) # shift the minimum value to 0 and increase sparseness
data['totmcst_log'] = np.log(data['totmcst'] + 1e-3)  # Adding a small constant to avoid log(0)
data['totmcst_log'] = 5*(data['totmcst_log']-min(data['totmcst_log'])) # shift the minimum value to 0 and increase sparseness
data[['charges_log', 'totcst_log', 'totmcst_log']].describe()

In [ ]:
# Vital Signs
# These are often important indicators of health status and can be used to predict outcomes.
data['low_BP'] = data['meanbp'] <= 65
data['high_HR'] = data['hrt'] >= 100
data['high_resp'] = data['resp'] >= 30
data['high_temp'] = data['temp'] >= 38.0
data['low_temp'] = data['temp'] <= 36.0
data[['low_BP', 'high_HR', 'high_resp', 'high_temp', 'low_temp']].describe()

In [ ]:
# pafi is a measure of the severity of illness, often used in critical care settings.
data['ARDS_severity'] = data['pafi'].apply(lambda x: 'Normal' if x >=300 else ('Mild' if x >= 200 else ('Moderate' if x >= 100 else 'Severe')))
data['ARDS_severity'].value_counts()

In [ ]:
# arterial ph
data['acidosis'] = data['ph'] < 7.35
data['alkalosis'] = data['ph'] > 7.45
data[['acidosis', 'alkalosis']].describe()

In [ ]:
# albumin
data['albumin_low'] = data['alb'] < 3.5
data['albumin_high'] = data['alb'] > 5.0
data[['albumin_low', 'albumin_high']].describe()

In [ ]:
# Bilirubin is a measure of liver function, often used in critical care settings.
data['bili_high'] = data['bili'] > 2.0
data[['bili_high']].describe()

In [ ]:
# Creatinine is a measure of kidney function, often used in critical care settings.
data['creatinine_high'] = data['crea'] > 2
data[['creatinine_high']].describe()

In [ ]:
# blood urea nitrogen (BUN) is a measure of kidney function, often used in critical care settings.
data['bun_high'] = data['bun'] > 20
# creatinine ratio is a measure of kidney function, often used in critical care settings.
data['creatinine_ratio'] = data['crea'] / (data['bun']+1e-6)  # Adding a small constant to avoid division by zero
data[['bun_high']].describe(),data[['creatinine_ratio']].describe()

In [ ]:
# urine output is a measure of kidney function, often used in critical care settings.
data['urine_output_low'] = data['urine'] < 500
data[['urine_output_low']].describe()

In [ ]:
# glucose is a measure of blood sugar levels, often used in critical care settings.
data['glucose_high'] = data['glucose'] > 200
data[['glucose_high']].describe()

In [ ]:
# adl scores
# 1. disability level
data['disability_level'] = data['adlsc'] <4
# 2. cognitive issues that leads to over optimism
data['cognitive_optimism'] = data['adlp']-data['adls']
# scaling to [0,1] range, showcasing a probability of cognitive optimism
data['cognitive_optimism'] = 1 / (1 + np.exp(-data['cognitive_optimism']))
data[['disability_level']].describe(), data[['cognitive_optimism']].describe()

In [ ]:
data.columns

In [ ]:
# Organ Failures
# Each organ failure indicator is boolean, so sum them to count organ failures per row
data['organ_failure'] = (
	data['low_BP'].astype(int) +
	data['creatinine_high'].astype(int) +
	data['urine_output_low'].astype(int) +
	data['acidosis'].astype(int)
)
data['organ_failure'] =  1.5 / (1 + np.exp(-data['organ_failure'])) - 0.5*1.5
data['organ_failure'].describe()

In [ ]:
data

In [ ]:
# delete columns with low feature importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
def delete_low_importance_features(data, target_cols, threshold=0.01):
    """
    Delete columns with low feature importance based on Random Forest Classifier.
    
    Parameters:
    -----------
    data : pd.DataFrame
        The input DataFrame containing features and target.
    target_cols : list
        The names of the target columns.
    threshold : float
        The minimum feature importance value to retain a feature.
        
    Returns:
    --------
    pd.DataFrame
        The DataFrame with low importance features removed.
    """
    oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X = data.drop(columns=target_cols)
    y = data[target_cols[0]]  # Use the first target column for feature importance

    # Encode categorical columns
    X_encoded = X.copy()
    cat_cols = X_encoded.select_dtypes(include=['object', 'category']).columns
    if len(cat_cols) > 0:
        X_encoded[cat_cols] = oe.fit_transform(X_encoded[cat_cols])

    # Replace inf/-inf with np.nan, then fill np.nan with column median
    X_encoded = X_encoded.replace([np.inf, -np.inf], np.nan)
    X_encoded = X_encoded.fillna(X_encoded.median(numeric_only=True))

    # Fit Random Forest Classifier
    model = RandomForestClassifier(n_estimators=500,criterion='entropy',random_state=42)
    model.fit(X_encoded, y)

    # Get feature importances
    importances = model.feature_importances_

    # Select features above the threshold
    selected_features = X_encoded.columns[importances > threshold]
    
    return data[selected_features.tolist() + target_cols], model.feature_importances_
# Only drop columns that exist in the DataFrame
target_cols = [col for col in ['death', 'sfdm2', 'hospdead'] if col in data_no_outliers.columns]
data_no_low_importance, importance = delete_low_importance_features(data_no_outliers, target_cols=target_cols, threshold=0.0025)

# plot feature importances for selected features only
selected_features = data_no_low_importance.drop(columns=['death']).columns

# Only drop columns that exist in the DataFrame for original_features
original_features = data_no_outliers.drop(columns=target_cols).columns

# Get importances for selected_features by matching their positions in original_features
selected_importances = [importance[original_features.get_loc(col)] for col in selected_features if col in original_features]

# Ensure selected_features and selected_importances have the same length
selected_features_plot = [col for col in selected_features if col in original_features]

plt.figure(figsize=(10, 6))
plt.barh(selected_features_plot, selected_importances)
plt.xlabel('Feature Importance')
plt.title('Feature Importances after Removing Low Importance Features')
plt.show()
print(f"Number of features after removing low importance features: {len(selected_features_plot)}")

In [ ]:
cat_cols = data_no_low_importance.select_dtypes(include=['object', 'category']).columns
cat_cols

In [ ]:
dataUnderstanding(data_no_low_importance)

In [ ]:
# Replace inf/-inf with np.nan before imputation
data_no_low_importance = data_no_low_importance.replace([np.inf, -np.inf], [1e6,1e-6])
data_no_low_importance = fill_missing_values_simplified(data_no_low_importance)
data_no_low_importance.isnull().sum()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder, OneHotEncoder
NominalEncoder_list = ['dzgroup', 'dzclass','ca', 'dnr','sex','race','Age_Class']
map_information = {'nominal':{},'ordinal':{}}
# check the number of unique values in each categorical column
for col in cat_cols:
    print(f"Number of unique values in {col}: {data_no_low_importance[col].nunique()}")

In [ ]:
# Ensure data_no_low_importance is defined
if 'data_no_low_importance' not in globals():
    # Only drop columns that exist in the DataFrame
    target_cols = [col for col in ['death', 'sfdm2', 'hospdead'] if col in data_no_outliers.columns]
    data_no_low_importance, importance = delete_low_importance_features(data_no_outliers, target_cols=target_cols, threshold=0.005)

# encode nominal encoder list as onehot encoder
for col in NominalEncoder_list:
    if col in data_no_low_importance.columns:
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded_col = ohe.fit_transform(data_no_low_importance[[col]])
        encoded_col_df = pd.DataFrame(encoded_col, columns=[f"{col}_{i}" for i in range(encoded_col.shape[1])], index=data_no_low_importance.index)
        data_no_low_importance = pd.concat([data_no_low_importance, encoded_col_df], axis=1)
        data_no_low_importance.drop(columns=[col], inplace=True)
        # record the mapping information
        map_information['nominal'][col] = ohe.categories_[0].tolist()
map_information

In [ ]:
# encode ordinal encoder list and numeric columns
for col in cat_cols:
    # Skip columns that have been one-hot encoded and dropped, or are not present
    if col in NominalEncoder_list or col not in data_no_low_importance.columns:
        continue
    if data_no_low_importance[col].nunique() <= 30 and col!="scoma":  # Ordinal encoding for categorical columns (all with <= 30 unique values after manual check)
        oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        data_no_low_importance[col] = oe.fit_transform(data_no_low_importance[[col]])
        # record the mapping information as a dictionary: number value -> original value
        map_information['ordinal'][col] = {i: v for i, v in enumerate(oe.categories_[0])}
    else:
        data_no_low_importance[col] = data_no_low_importance[col].astype(float)  # Convert to float since mostly stated as float in the codebook
print(map_information)
# Only show columns that still exist
remaining_cat_cols = [col for col in cat_cols if col in data_no_low_importance.columns]
data_no_low_importance[remaining_cat_cols]

# Data Visualization

In [ ]:
df=data_no_low_importance
# use LDA to pick out important features
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Identify the correct target column for LDA
target_col = df['sfdm2']

# Prepare X and y for LDA
df = df.drop(columns=['sfdm2'])
X_lda_input = df.select_dtypes('number').copy()
y_lda = target_col

# LDA: n_components must be <= min(n_features, n_classes - 1)
n_classes = len(np.unique(y_lda))
n_features = X_lda_input.shape[1]
max_components = min(n_features, n_classes - 1)
lda = LDA(solver='svd',n_components=max_components)
X_lda = lda.fit_transform(X_lda_input, y_lda)
# Convert X_lda to DataFrame before concatenation
X_lda_df = pd.DataFrame(X_lda)
lda_df = pd.concat([X_lda_df], axis=1)
lda_df

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# 2D t-SNE plot
indexes = df[5000:].index
data_1000 = lda_df.iloc[indexes]
labels_1000 = target_col.iloc[indexes]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_1000)
tsne = TSNE(n_components=2, perplexity=170, random_state=3407, max_iter=10000)
tsne_result = tsne.fit_transform(scaled_data)
tsne_df = pd.DataFrame(data=tsne_result, columns=["Dim_1", "Dim_2"])
# Convert float labels to int for mapping
label_map = map_information['ordinal']['sfdm2']
tsne_df["label"] = [label_map.get(int(l), str(l)) for l in labels_1000.values]

sns.scatterplot(data=tsne_df, x='Dim_1', y='Dim_2',
               hue='label', palette="bright")
plt.show()

In [ ]:
# 3D t-SNE plot
tsne_3D = TSNE(n_components=3, perplexity=170, random_state=3407, max_iter=10000)
tsne_result_3D = tsne_3D.fit_transform(scaled_data)
tsne_df_3D = pd.DataFrame(data=tsne_result_3D, columns=["Dim_1", "Dim_2", "Dim_3"])
# Convert float labels to int for mapping
tsne_df_3D["label"] = [label_map.get(int(l), str(l)) for l in labels_1000.values]

# Map string labels to integers for coloring
unique_labels = tsne_df_3D["label"].unique()
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
tsne_df_3D["label_int"] = tsne_df_3D["label"].map(label_to_int)

from matplotlib.colors import ListedColormap, BoundaryNorm

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Create a ListedColormap for consistent color mapping
cmap = plt.get_cmap("Set1", len(unique_labels))
norm = BoundaryNorm(range(len(unique_labels) + 1), cmap.N)

scatter = ax.scatter(
	tsne_df_3D["Dim_1"], tsne_df_3D["Dim_2"], tsne_df_3D["Dim_3"],
	c=tsne_df_3D["label_int"], cmap=cmap, norm=norm, alpha=1, edgecolors='white'
)

# Create legend with matching colors
handles = [
	plt.Line2D([0], [0], marker='o', color='w', label=label,
			   markerfacecolor=cmap(idx), markersize=10)
	for label, idx in label_to_int.items()
]
ax.legend(handles=handles, title="label")
plt.show()

In [ ]:
# Clustering Analysis
df = data_no_low_importance
df.to_csv("preprocessed_data.csv",index=False)

# Prediction: Training and Testing

# Evaluation and Choice of Prediction Model

# Open-Ended Exploration